# Baseline

In [1]:
# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
#from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

C:\Program Files\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Program Files\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train_df = pd.read_json("../input/train.json")
test_df = pd.read_json("../input/test.json")

In [115]:
y=train_df['interest_level']
y2 = []
for k in y:
    if k == 'low': 
        y2.append(1)
    elif k == 'medium': 
        y2.append(2)
    elif k == 'high':
        y2.append(3)
    else: 
        y2.append(-1) 

y2=pd.DataFrame(y2)

#change so they are ordered 

X = train_df.drop('interest_level', 1)

from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(
    X, y2, test_size=0.33, random_state=1)

print('train: {} dev: {}'.format(X_train.shape, X_dev.shape))
print('train: {} dev: {}'.format(y_train.shape, y_dev.shape))

#X_train.head()
#print(y[0:3], y2[0:3], y_train.head())

train: (33065, 14) dev: (16287, 14)
train: (33065, 1) dev: (16287, 1)


In [116]:
def txt_features(dat):
    strlen = [len(x) for x in (dat['description'])]
    numwords = [len(x.split()) for x in (dat['description'])]
    numcaps = [sum(1 for c in x if c.upper()) for x in (dat['description'])]
    numpunct = [sum(1 for c in x if c in punctuation) for x in (dat['description'])]
    richness = [len(set(x)) / (len(x)+0.001) for x in dat['description']]

    ddf = pd.DataFrame()
    ddf['strlen'] = strlen
    ddf['numwords'] = numwords
    ddf['numcaps'] = numcaps
    ddf['numpunct'] = numpunct
    ddf['richness'] = richness
    return ddf

In [117]:
from sklearn.ensemble import RandomForestClassifier
 
ddf_tr = txt_features(X_train)
ddf_de = txt_features(X_dev)

rfc = RandomForestClassifier(n_estimators=10, n_jobs=-1) #-1 means use all available cores
rfc.fit(ddf_tr, y_train)
print('Training Score: %.2f%%' % (rfc.score(ddf_tr, y_train) * 100))

rfc.fit(ddf_de, y_dev)
print('Dev Score: %.2f%%' % (rfc.score(ddf_de, y_dev) * 100))


C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Training Score: 93.43%


C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Dev Score: 94.56%


In [118]:
importances = rfc.feature_importances_
features = ddf_tr.columns

sort_indices = np.argsort(importances)[::-1]
sorted_features = []
for i in sort_indices:
    sorted_features.append(features[i])

vals = importances[sort_indices]
labels = sorted_features

print labels
print vals

['richness', 'strlen', 'numcaps', 'numwords', 'numpunct']
[ 0.25651105  0.18787878  0.1868646   0.1863191   0.18242646]
